<a href="https://colab.research.google.com/github/Adem-HB/Adem-HB/blob/main/scarping_avis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import pdb
import re
import os
import random
from bs4 import BeautifulSoup
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
headers = {"User-Agent": "Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/115.0"}
base_url = 'https://fr.trustpilot.com/review/www.cdiscount.com'

In [ ]:
def fetch_url(url, headers, max_attempts=10):
    for attempt in range(max_attempts):
        timeout = random.randint(1, 10)
        try:
            response = requests.get(url, headers=headers, timeout=timeout)
            response.raise_for_status()  # Raise en exception pour les HTTP erreurs
            print(f'Successfully fetched {url}')
            return response
        except requests.RequestException as e:
            print(f'Attempt {attempt + 1} failed for {url} with timeout {timeout}: {e}')
    return None

In [ ]:
response = fetch_url(base_url, headers)
contenu = response.text

Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com


In [ ]:
def extract_comments_and_total(contenu):
    comments = re.findall(r'"reviewBody":"(.*?)"', contenu)
    rating = re.findall(r'"ratingValue":"(.*?)"', contenu)
    nb_views_match = re.search(r"(?:[\d\s]+) sur ([\d\s]+) avis", contenu)
    total_reviews = int(nb_views_match.group(1).replace('\u202f', '').replace(' ', '')) if nb_views_match else len(comments)
    return comments, rating, total_reviews



In [ ]:
import math
import re

comments, ratings, total_reviews = extract_comments_and_total(contenu)

all_data = [{"comment": c, "rating": float(r)} for c, r in zip(comments, ratings)]

# Calcul du nombre total de pages
total_pages = math.ceil(total_reviews / 20) if total_reviews else 1
print(f"Nombre total d'avis : {total_reviews} => {total_pages} pages estimées.")

# Récupération des pages suivantes
for page in range(2, 10 + 1):
    url = f"{base_url}?page={page}"
    response = fetch_url(url, headers)  # 👈 tu dois refetch le contenu de la page !
    contenu = response.text

    page_comments = re.findall(r'"reviewBody":"(.*?)"', contenu)
    page_ratings = re.findall(r'"reviewRating":{"@type":"Rating","bestRating":"5","worstRating":"1","ratingValue":"(.*?)"', contenu)

    if not page_comments:
        print(f"Aucun commentaire trouvé à la page {page}, arrêt.")
        break

    all_data.extend(
        {"comment": c, "rating": float(r)}
        for c, r in zip(page_comments, page_ratings)
    )
    print(f"Page {page} : {len(page_comments)} commentaires trouvés.")

print(f"\nTotal de commentaires récupérés : {len(all_data)}")


Nombre total d'avis : 267494 => 13375 pages estimées.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=2
Page 2 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=3
Page 3 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=4
Page 4 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=5
Page 5 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=6
Page 6 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=7
Page 7 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=8
Page 8 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot.com/review/www.cdiscount.com?page=9
Page 9 : 20 commentaires trouvés.
Successfully fetched https://fr.trustpilot

In [ ]:
all_data

[{'comment': 'J’ai commandé une lave vaisselle après 3 jours ils ont augmenter le prix et ils ont annulés ma commande.C’est le pire service que j’ai eu dans ma vie',
  'rating': 2.7},
 {'comment': "Très déçu du site. Je n'ai jamais osé passer de commande.J'ai voulais passer ma 1ere commande hier soir (efin) et m'acheter une machine à café.  Je n'ai malheureusement pas pu valider ma commande car problème de connexion sur le site et j'ai bloqué ma CB.Ce matin après avoir débloqué ma carte aupres de ma banque, je souhaite valider ma commande et grande surprise je vois que la machine à augmenté de plus de 150 euros !!!!!!!J'ai tenté de joindre le service client pour expliquer que le problème est indépendant de ma volonté, mais rien à faire. Ils sont de très mauvaise foisJe ne commanderai pas chez eux, maintenant c'est certain.Pour un nouveau client ce n'est pas pro du tout !!!!!!Jattend de leur nouvelle et un effort de leur part",
  'rating': 1.0},
 {'comment': "En réponse au service sav d

In [ ]:
from transformers import pipeline
from sklearn.metrics import classification_report


# Pipeline pré-entraîné en sentiment

classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment", tokenizer="nlptown/bert-base-multilingual-uncased-sentiment", truncation=True)



/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/953 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/669M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/872k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Device set to use cpu


**Modele 1 tester**

In [ ]:
# === Modèle 1 : nlptown (multilingue prêt à l’emploi) ===
# Convertir en DataFrame
df = pd.DataFrame(all_data)
df_filtered = df.copy()
df_filtered["comment_len"] = df_filtered["comment"].apply(lambda x: len(x.split()))
print(df_filtered["comment_len"].describe())
print(df_filtered[df_filtered["comment_len"] > 512]["comment"].head())

count    200.00000
mean      81.31000
std       69.50734
min        8.00000
25%       38.00000
50%       62.00000
75%      103.25000
max      508.00000
Name: comment_len, dtype: float64
Series([], Name: comment, dtype: object)


In [ ]:
# 1. Fonction pour mapper les ratings réels vers des classes
def rating_to_label(score):
    if score < 3.5:
        return "negatif"
    else:
        return "positif"

df_filtered["true_label"] = df_filtered["rating"].apply(rating_to_label)

# 3. Fonction pour mapper les labels du modèle vers les mêmes classes
def map_output_nlptown(label):
    if "1" in label or "2" in label or "3" in label:
        return "negatif"
    else:
        return "positif"

MAX_WORDS = 200  # facultatif : réduire aussi côté mots

def truncate_text(text, max_words=MAX_WORDS):
    return " ".join(text.split()[:max_words])

# Tronquer les textes (optionnel mais recommandé)
df_filtered["comment_trunc"] = df_filtered["comment"].apply(truncate_text)

# Appliquer le classifieur en lot
results = classifier(df_filtered["comment_trunc"].tolist(), batch_size=32)

# Mapper les résultats vers les labels
df_filtered["pred_label"] = [map_output_nlptown(res["label"]) for res in results]

# Mapper les labels réels
df_filtered["true_label"] = df_filtered["rating"].apply(rating_to_label)

# Rapport de classification
print(classification_report(df_filtered["true_label"], df_filtered["pred_label"], digits=3))

              precision    recall  f1-score   support

     negatif      0.000     0.000     0.000       190
     positif      0.050     1.000     0.095        10

    accuracy                          0.050       200
   macro avg      0.025     0.500     0.048       200
weighted avg      0.003     0.050     0.005       200



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


**Model 2**

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, pipeline

model_name = "cardiffnlp/twitter-xlm-roberta-base-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)

# Pipeline avec troncation
classifier_model_2 = pipeline(
    "sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    truncation=True
)

# 1. Appliquer la prédiction
df_filtered["pred_label"] = df_filtered["comment"].apply(
    lambda x: classifier_model_2(x)[0]["label"]
)

# 2. Uniformiser les labels
def map_cardiff_label(label):
    return {
        "negative": "negatif",
        "neutral": "neutre",
        "positive": "positif"
    }[label]

df_filtered["pred_label"] = df_filtered["pred_label"].apply(map_cardiff_label)

# 3. Affichage final
print(df_filtered["pred_label"])


Device set to use cpu


KeyError: 'neutral'

In [ ]:
# 2. Uniformiser les labels
def map_cardiff_label(label):
    return {
        "negative": "negatif",
        "neutral": "neutre",
        "positive": "positif"
    }[label]

In [ ]:
df_filtered["true_label"] = df_filtered["rating"].apply(rating_to_label)
print(classification_report(df_filtered["true_label"], df_filtered["pred_label"], digits=3))

              precision    recall  f1-score   support

     negatif      0.944     0.918     0.931       183
      neutre      0.000     0.000     0.000         7
     positif      0.353     0.600     0.444        10

    accuracy                          0.870       200
   macro avg      0.432     0.506     0.458       200
weighted avg      0.881     0.870     0.874       200



In [ ]:

# Appliquer
def map_allocine(label):
    return "positif" if label == "POS" else "negatif"

df_filtered["pred_label"] = df_filtered["comment"].apply(lambda x: map_allocine(classifier_model_3(x, truncation=True)[0]["label"]))


Device set to use cpu


In [ ]:
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import pandas as pd

# 1. Charger le bon modèle et tokenizer
classifier_model_3 = pipeline("sentiment-analysis", model="tblard/tf-allocine", tokenizer=tokenizer, max_length=512, truncation=True)

# 2. Mapper les labels du modèle vers vos classes
def map_allocine(label):
    return "positif" if label == "POS" else "negatif"

# 3. Appliquer le classifieur
df_filtered["pred_label"] = df_filtered["comment"].apply(
    lambda x: map_allocine(classifier_model_3(x)[0]["label"])
)

# 4. S'assurer que `true_label` est bien présent
df_filtered["true_label"] = df_filtered["rating"].apply(rating_to_label)

# 5. Créer le DataFrame de résultat
result_df = df_filtered[["rating", "true_label", "pred_label"]]

# 6. (Optionnel) Affichage lisible
pd.set_option('display.max_colwidth', 150)

# 7. Afficher les résultats
print(result_df)


All model checkpoint layers were used when initializing TFCamembertForSequenceClassification.

All the layers of TFCamembertForSequenceClassification were initialized from the model checkpoint at tblard/tf-allocine.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFCamembertForSequenceClassification for predictions without further training.
Device set to use 0


     rating true_label pred_label
0       2.7     neutre    negatif
1       1.0    negatif    negatif
2       1.0    negatif    negatif
3       1.0    negatif    negatif
4       2.0    negatif    negatif
..      ...        ...        ...
195     1.0    negatif    negatif
196     1.0    negatif    negatif
197     1.0    negatif    negatif
198     1.0    negatif    negatif
199     1.0    negatif    negatif

[200 rows x 3 columns]


In [ ]:
df_filtered["true_label"] = df_filtered["rating"].apply(rating_to_label)
print(classification_report(df_filtered["true_label"], df_filtered["pred_label"], digits=3))

              precision    recall  f1-score   support

     negatif      0.915     1.000     0.956       183
      neutre      0.000     0.000     0.000         7
     positif      0.000     0.000     0.000        10

    accuracy                          0.915       200
   macro avg      0.305     0.333     0.319       200
weighted avg      0.837     0.915     0.874       200



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
